In [ ]:
# Montar o google drive no coolab
from google.colab import drive
drive.mount('/content/drive')

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from collections import Counter
from itertools import chain

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from transformers import BertTokenizer, RobertaTokenizer, BertTokenizerFast, BertModel, BertForSequenceClassification
import torch
import torch.nn.functional as F
import torch.nn as nn

from sklearn.metrics import classification_report
from sklearn import metrics

import pickle
import re
import os
from tqdm import tqdm
import ast
import random
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset


## Importando DataFrames

In [ ]:
df_test_GPTrelev = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_GPTrelev_MinTL.pkl')
df_test_w2vreglog = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_w2vreglog_MinTL.pkl')
df_test_softbert = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_softBERT_MinTL.pkl')

In [ ]:
# Confusion matrix
cm = confusion_matrix(df_test_GPTrelev['Numeric_Diagnosed_YN'], df_test_GPTrelev['predict_Diagnosed_YN'])

# Exibindo a matriz de confusão
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Renomear colunas no df_test_GPTrelev
df_test_GPTrelev = df_test_GPTrelev.rename(columns={'predict_Diagnosed_YN': 'predict_GPTrelev', 'qt_msgs_lidas': 'qtd_msgs_lidas_GPTrelev'})

# Renomear colunas no df_test_w2vreglog
df_test_w2vreglog = df_test_w2vreglog.rename(columns={'prediction': 'predict_w2vreglog', 'qtd_msgs_lidas': 'qtd_msgs_lidas_w2vreglog'})

# Renomear colunas no df_test_softbert
df_test_softbert = df_test_softbert.rename(columns={'prediction': 'predict_softbert', 'qtd_msgs_lidas': 'qtd_msgs_lidas_softbert'})

In [ ]:
# Unir df_test_w2vreglog e df_test_softbert
df_merged = pd.merge(df_test_w2vreglog[['User_ID', 'Diagnosed_YN', 'predict_w2vreglog', 'qtd_msgs_lidas_w2vreglog']],
                     df_test_softbert[['User_ID', 'predict_softbert', 'qtd_msgs_lidas_softbert']],
                     on='User_ID',
                     how='inner')

# Unir o resultado com df_test_GPTrelev
df_final = pd.merge(df_merged,
                    df_test_GPTrelev[['User_ID', 'label_min', 'predict_GPTrelev', 'qtd_msgs_lidas_GPTrelev']],
                    on='User_ID',
                    how='inner')


In [ ]:
df_final['Erro_w2vreglog'] = (df_final['predict_w2vreglog'] != df_final['Diagnosed_YN']).astype(int)
df_final['Erro_softbert'] = (df_final['predict_softbert'] != df_final['Diagnosed_YN']).astype(int)
df_final['Erro_GPTrelev'] = (df_final['predict_GPTrelev'] != df_final['Diagnosed_YN']).astype(int)

In [ ]:
df_final.to_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/analise_erro.pkl')
# df_final = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/analise_erro.pkl')
df_final.head()

## Análise de erros

1. Selecionar aleatoriamente N instâncias de teste positivas e N negativas que tenham sido incorretamente classificadas; se possível, considere N como o nro total de instâncias positivas incorretas do conjunto de teste
2. Registrar a quantidade (%) dessas instâncias que o baseline errou também, e descartar
3. Considerando apenas o restante, ou seja, instâncias que o GPTrelev classificou errado e o baseline certo

In [ ]:
df_final.value_counts('predict_GPTrelev', dropna=False)

In [ ]:
df_final.value_counts('Diagnosed_YN', dropna=False)

In [ ]:
# selecionar aleatoriamente N instâncias positivas e N negativas que tenham sido incorretamente classificadas
# N = total de instâncias positivas incorretas

qtd_positivas = len(df_final[(df_final['predict_GPTrelev'] == 0) & (df_final['Diagnosed_YN'] == 1)]) # instâncias positivas classificadas incorretamente
qtd_negativas = len(df_final[(df_final['predict_GPTrelev'] == 1) & (df_final['Diagnosed_YN'] == 0)]) # instâncias negativas classificadas incorretamente

print(f'Quantidade de instâncias positivas classificadas incorretamente: {qtd_positivas}')
print(f'Quantidade de instâncias negativas classificadas incorretamente: {qtd_negativas}')

In [ ]:
# selecionar aleatoriamente N instâncias positivas e N negativas que tenham sido incorretamente classificadas
# N = total de instâncias positivas incorretas

qtd_positivas = len(df_final[(df_final['predict_GPTrelev'] == 0) & (df_final['Diagnosed_YN'] == 1) & (df_final['predict_w2vreglog'] == 1)]) # instâncias positivas classificadas incorretamente
qtd_negativas = len(df_final[(df_final['predict_GPTrelev'] == 1) & (df_final['Diagnosed_YN'] == 0) & (df_final['predict_w2vreglog'] == 0)]) # instâncias negativas classificadas incorretamente

print(f'Quantidade de instâncias positivas classificadas incorretamente com acerto do baseline: {qtd_positivas}')
print(f'Quantidade de instâncias negativas classificadas incorretamente com acerto do baseline: {qtd_negativas}')

### Intersecções

In [ ]:
# selecionar aleatoriamente N instâncias positivas e N negativas que tenham sido incorretamente classificadas
# N = total de instâncias positivas incorretas

qtd_concordancia_positivas = len(df_final[(df_final['predict_GPTrelev'] == 1) & (df_final['predict_w2vreglog'] == 1)])
qtd_concordancia_negativas = len(df_final[(df_final['predict_GPTrelev'] == 0) & (df_final['predict_w2vreglog'] == 0)])
qtd_discordancia_positivas = len(df_final[(df_final['predict_GPTrelev'] == 1) & (df_final['predict_w2vreglog'] == 0)])
qtd_discordancia_negativas = len(df_final[(df_final['predict_GPTrelev'] == 0) & (df_final['predict_w2vreglog'] == 1)])

print(f'Quantidade de concordâncias positivas classificadas: {qtd_concordancia_positivas}')
print(f'Quantidade de concordâncias negativas classificadas: {qtd_concordancia_negativas}')
print(f'Quantidade de discordâncias positivas classificadas: {qtd_discordancia_positivas}')
print(f'Quantidade de discordâncias negativas classificadas: {qtd_discordancia_negativas}')

### Métricas de desempenho

In [ ]:
# GPTrelev
y_test = df_final['Diagnosed_YN']
y_pred = df_final['predict_GPTrelev']
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

print(' ')
print(classification_report(y_test, y_pred))

In [ ]:
# W2V.logreg
y_test = df_final['Diagnosed_YN']
y_pred = df_final['predict_w2vreglog']
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

print(' ')
print(classification_report(y_test, y_pred))

In [ ]:
# Soft.BERT
y_test = df_final['Diagnosed_YN']
y_pred = df_final['predict_softbert']
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

print(' ')
print(classification_report(y_test, y_pred))

### Distribuição das notas

In [ ]:
# Funções

# Substituição de texto por zero
def replace_phrases_with_zero(lst):
    new_lst = [0 if isinstance(item, str) and not item.isdigit() else int(item) for item in lst]
    return new_lst

# Obtenção da lista de notas
def escore_list(df):
  notas_list = list(chain.from_iterable(df['label_min']))
  notas_list_filtrada = [nota for nota in notas_list if nota not in (0, 1, 2)]

  # Contar a frequência dos valores
  contagens = Counter(notas_list_filtrada)

  # Criar um DataFrame a partir do dicionário
  df_contagens = pd.DataFrame(contagens.items(), columns=['notas', 'quantidade'])

  # Calcular o total de todas as quantidades
  total = df_contagens['quantidade'].sum()

  # Adicionar a coluna de percentual
  df_contagens['percentual'] = round((df_contagens['quantidade'] / total) * 100, 2)
  df_contagens.sort_values('notas', inplace=True)

  return df_contagens

# Obtenção da lista de notas
def escore_list_n_relev(df):
  notas_list = list(chain.from_iterable(df['label_min']))
  notas_list_filtrada = [nota for nota in notas_list if nota in (0, 1, 2)]

  # Contar a frequência dos valores
  contagens = Counter(notas_list_filtrada)

  # Criar um DataFrame a partir do dicionário
  df_contagens = pd.DataFrame(contagens.items(), columns=['notas', 'quantidade'])

  # Calcular o total de todas as quantidades
  total = df_contagens['quantidade'].sum()

  # Adicionar a coluna de percentual
  df_contagens['percentual'] = round((df_contagens['quantidade'] / total) * 100, 2)
  df_contagens.sort_values('notas', inplace=True)

  return df_contagens

# Obtenção da lista de notas
def escore_list_total(df):
  notas_list = list(chain.from_iterable(df['label_min']))

  # Contar a frequência dos valores
  contagens = Counter(notas_list)

  # Criar um DataFrame a partir do dicionário
  df_contagens = pd.DataFrame(contagens.items(), columns=['notas', 'quantidade'])

  # Calcular o total de todas as quantidades
  total = df_contagens['quantidade'].sum()

  # Adicionar a coluna de percentual
  df_contagens['percentual'] = round((df_contagens['quantidade'] / total) * 100, 2)
  df_contagens.sort_values('notas', inplace=True)

  return df_contagens

In [ ]:
df_final['label_min'] = df_final['label_min'].apply(replace_phrases_with_zero)

# Dataframes de erros e acertos
df_final_erros = df_final[(df_final['Erro_GPTrelev'] == 1)]
df_final_acertos = df_final[(df_final['Erro_GPTrelev'] == 0)]

# Serparação nas classes positiva e negativa
df_final_erros_positiva = df_final_erros[df_final_erros['predict_GPTrelev'] == 1]
df_final_erros_negativa = df_final_erros[df_final_erros['predict_GPTrelev'] == 0]

df_final_acertos_positiva = df_final_acertos[df_final_acertos['predict_GPTrelev'] == 1]
df_final_acertos_negativa = df_final_acertos[df_final_acertos['predict_GPTrelev'] == 0]

print('Acertos da classe positiva')
df_escores_acertos_positiva = escore_list(df_final_acertos_positiva)
print(df_escores_acertos_positiva)
relevantes_acertos_positiva = df_escores_acertos_positiva['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_acertos_positiva}')

print('##############################')

print('Acertos da classe negativa')
df_escores_acertos_negativa = escore_list(df_final_acertos_negativa)
print(df_escores_acertos_negativa)
relevantes_acertos_negativa = df_escores_acertos_negativa['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_acertos_negativa}')

print('##############################')

print('Erros da classe positiva')
df_escores_erros_positiva = escore_list(df_final_erros_positiva)
print(df_escores_erros_positiva)
relevantes_erros_positiva = df_escores_erros_positiva['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_erros_positiva}')

print('##############################')

print('Erros da classe negativa')
df_escores_erros_negativa = escore_list(df_final_erros_negativa)
print(df_escores_erros_negativa)
relevantes_erros_negativa = df_escores_erros_negativa['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_erros_negativa}')

In [ ]:
df_final['label_min'] = df_final['label_min'].apply(replace_phrases_with_zero)

# Dataframes de erros e acertos
df_final_erros = df_final[(df_final['Erro_GPTrelev'] == 1)]
df_final_acertos = df_final[(df_final['Erro_GPTrelev'] == 0)]

# Serparação nas classes positiva e negativa
df_final_erros_positiva = df_final_erros[df_final_erros['predict_GPTrelev'] == 1]
df_final_erros_negativa = df_final_erros[df_final_erros['predict_GPTrelev'] == 0]

df_final_acertos_positiva = df_final_acertos[df_final_acertos['predict_GPTrelev'] == 1]
df_final_acertos_negativa = df_final_acertos[df_final_acertos['predict_GPTrelev'] == 0]

print('Acertos da classe positiva')
df_escores_acertos_positiva = escore_list_n_relev(df_final_acertos_positiva)
print(df_escores_acertos_positiva)
relevantes_acertos_positiva = df_escores_acertos_positiva['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_acertos_positiva}')

print('##############################')

print('Acertos da classe negativa')
df_escores_acertos_negativa = escore_list_n_relev(df_final_acertos_negativa)
print(df_escores_acertos_negativa)
relevantes_acertos_negativa = df_escores_acertos_negativa['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_acertos_negativa}')

print('##############################')

print('Erros da classe positiva')
df_escores_erros_positiva = escore_list_n_relev(df_final_erros_positiva)
print(df_escores_erros_positiva)
relevantes_erros_positiva = df_escores_erros_positiva['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_erros_positiva}')

print('##############################')

print('Erros da classe negativa')
df_escores_erros_negativa = escore_list_n_relev(df_final_erros_negativa)
print(df_escores_erros_negativa)
relevantes_erros_negativa = df_escores_erros_negativa['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_erros_negativa}')

In [ ]:
df_final['label_min'] = df_final['label_min'].apply(replace_phrases_with_zero)

# Dataframes de erros e acertos
df_final_erros = df_final[(df_final['Erro_GPTrelev'] == 1)]
df_final_acertos = df_final[(df_final['Erro_GPTrelev'] == 0)]

# Serparação nas classes positiva e negativa
df_final_erros_positiva = df_final_erros[df_final_erros['predict_GPTrelev'] == 1]
df_final_erros_negativa = df_final_erros[df_final_erros['predict_GPTrelev'] == 0]

df_final_acertos_positiva = df_final_acertos[df_final_acertos['predict_GPTrelev'] == 1]
df_final_acertos_negativa = df_final_acertos[df_final_acertos['predict_GPTrelev'] == 0]

print('Acertos da classe positiva')
df_escores_acertos_positiva = escore_list_total(df_final_acertos_positiva)
print(df_escores_acertos_positiva)
relevantes_acertos_positiva = df_escores_acertos_positiva['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_acertos_positiva}')

print('##############################')

print('Acertos da classe negativa')
df_escores_acertos_negativa = escore_list_total(df_final_acertos_negativa)
print(df_escores_acertos_negativa)
relevantes_acertos_negativa = df_escores_acertos_negativa['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_acertos_negativa}')

print('##############################')

print('Erros da classe positiva')
df_escores_erros_positiva = escore_list_total(df_final_erros_positiva)
print(df_escores_erros_positiva)
relevantes_erros_positiva = df_escores_erros_positiva['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_erros_positiva}')

print('##############################')

print('Erros da classe negativa')
df_escores_erros_negativa = escore_list_total(df_final_erros_negativa)
print(df_escores_erros_negativa)
relevantes_erros_negativa = df_escores_erros_negativa['quantidade'].sum()
print(f'Quantidade de mensagens relevantes: {relevantes_erros_negativa}')

### Elapsed

In [ ]:
# Import e criação do dataframe para análise
df_elapsed = pd.read_excel('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/elapsed_times.xlsx', sheet_name='diagnosed_test_anxiety')


df_merged = pd.merge(df_final, df_elapsed, on='User_ID', how='left')
df_merged.head()

In [ ]:
df_merged.pivot_table(index='Erro_GPTrelev', columns='Old', aggfunc='count', values='Diagnosed_YN')

### H1 - Timeline negativa apresenta indícios de ansiedade (Falsos positivos)

Distribuição de notas para os casos de erro

In [ ]:
def replace_phrases_with_zero(lst):
    new_lst = [0 if isinstance(item, str) and not item.isdigit() else int(item) for item in lst]
    return new_lst

df_final['label_min'] = df_final['label_min'].apply(replace_phrases_with_zero)

# Distribuição das notas nestas timelines
df_final_erros = df_final[(df_final['predict_GPTrelev'] == 1) & (df_final['Diagnosed_YN'] == 0) & (df_final['predict_w2vreglog'] == 0)]

In [ ]:
notas_list_erros = list(chain.from_iterable(df_final_erros['label_min']))
notas_list_erros_filtrada = [nota for nota in notas_list_erros if nota not in (0, 1, 2)]

# Contar a frequência dos valores
contagens_erros = Counter(notas_list_erros_filtrada)

# Criar um DataFrame a partir do dicionário
df_contagens_erros = pd.DataFrame(contagens_erros.items(), columns=['notas', 'quantidade'])

# Calcular o total de todas as quantidades
total = df_contagens_erros['quantidade'].sum()

# Adicionar a coluna de percentual
df_contagens_erros['percentual'] = round((df_contagens_erros['quantidade'] / total) * 100, 2)
df_contagens_erros.sort_values('notas')

In [ ]:
# Média geral
round(sum(notas_list_erros_filtrada)/len(notas_list_erros_filtrada), 2)

Distribuição de notas para a classe positiva

In [ ]:
df_final_positivo = df_final[df_final['Diagnosed_YN'] == 1]

notas_list_pos = list(chain.from_iterable(df_final_positivo['label_min']))
notas_list_pos_filtrada = [nota for nota in notas_list_pos if nota not in (0, 1, 2)]

# Contar a frequência dos valores
contagens_pos = Counter(notas_list_pos_filtrada)

# Criar um DataFrame a partir do dicionário
df_contagens_pos = pd.DataFrame(contagens_pos.items(), columns=['notas', 'quantidade'])

# Calcular o total de todas as quantidades
total = df_contagens_pos['quantidade'].sum()

# Adicionar a coluna de percentual
df_contagens_pos['percentual'] = round((df_contagens_pos['quantidade'] / total) * 100, 2)
df_contagens_pos.sort_values('notas')

In [ ]:
# Média geral
round(sum(notas_list_pos_filtrada)/len(notas_list_pos_filtrada), 2)

### H2 - Distribuição de notas para a matriz de confusão

In [ ]:
df_final_TP = df_final[(df_final['predict_GPTrelev'] == 1) & (df_final['Diagnosed_YN'] == 1)]
df_final_TN = df_final[(df_final['predict_GPTrelev'] == 0) & (df_final['Diagnosed_YN'] == 0)]
df_final_FP = df_final[(df_final['predict_GPTrelev'] == 1) & (df_final['Diagnosed_YN'] == 0)]
df_final_FN = df_final[(df_final['predict_GPTrelev'] == 0) & (df_final['Diagnosed_YN'] == 1)]

In [ ]:
dataframe_list = ['TP', 'TN', 'FP', 'FN']

In [ ]:
for i in dataframe_list:
  print(i)
  notas_list = list(chain.from_iterable(globals()[f'df_final_{i}']['label_min']))

  # Contar a frequência dos valores
  contagens = Counter(notas_list)

  # Criar um DataFrame a partir do dicionário
  globals()[f'df_contagens_{i}'] = pd.DataFrame(contagens.items(), columns=['notas', 'quantidade'])

  # Calcular o total de todas as quantidades
  total = globals()[f'df_contagens_{i}']['quantidade'].sum()

  # Adicionar a coluna de percentual
  globals()[f'df_contagens_{i}']['percentual'] = round((globals()[f'df_contagens_{i}']['quantidade'] / total) * 100, 2)
  print(globals()[f'df_contagens_{i}'].sort_values('notas'))
  print('--------------------------------------')

### H3 - Timeline positiva não apresenta indícios de ansiedade (Falsos negativos)

Distribuição de notas para os casos de erro

In [ ]:
def replace_phrases_with_zero(lst):
    new_lst = [0 if isinstance(item, str) and not item.isdigit() else int(item) for item in lst]
    return new_lst

df_final['label_min'] = df_final['label_min'].apply(replace_phrases_with_zero)

# Distribuição das notas nestas timelines
df_final_erros = df_final[(df_final['predict_GPTrelev'] == 0) & (df_final['Diagnosed_YN'] == 1) & (df_final['predict_w2vreglog'] == 1)]

In [ ]:
notas_list_erros = list(chain.from_iterable(df_final_erros['label_min']))
notas_list_erros_filtrada = [nota for nota in notas_list_erros if nota not in (0, 1, 2)]

# Contar a frequência dos valores
contagens_erros = Counter(notas_list_erros_filtrada)

# Criar um DataFrame a partir do dicionário
df_contagens_erros = pd.DataFrame(contagens_erros.items(), columns=['notas', 'quantidade'])

# Calcular o total de todas as quantidades
total = df_contagens_erros['quantidade'].sum()

# Adicionar a coluna de percentual
df_contagens_erros['percentual'] = round((df_contagens_erros['quantidade'] / total) * 100, 2)
df_contagens_erros.sort_values('notas')

In [ ]:
# Média geral
round(sum(notas_list_erros_filtrada)/len(notas_list_erros_filtrada), 2)

Distribuição de notas para a classe negativa

In [ ]:
df_final_negativo = df_final[df_final['Diagnosed_YN'] == 0]

notas_list_pos = list(chain.from_iterable(df_final_negativo['label_min']))
notas_list_pos_filtrada = [nota for nota in notas_list_pos if nota not in (0, 1, 2)]

# Contar a frequência dos valores
contagens_pos = Counter(notas_list_pos_filtrada)

# Criar um DataFrame a partir do dicionário
df_contagens_pos = pd.DataFrame(contagens_pos.items(), columns=['notas', 'quantidade'])

# Calcular o total de todas as quantidades
total = df_contagens_pos['quantidade'].sum()

# Adicionar a coluna de percentual
df_contagens_pos['percentual'] = round((df_contagens_pos['quantidade'] / total) * 100, 2)
df_contagens_pos.sort_values('notas')

In [ ]:
# Média geral
round(sum(notas_list_pos_filtrada)/len(notas_list_pos_filtrada), 2)